<a href="https://colab.research.google.com/github/HodaMemar/Patient-Similarity-through-Representation/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# **Preprocessing Unstructured data**

Step 1: Detect sections in Clinical Notes


In [ ]:
from google.colab import drive
drive.mount('/content/mydrive/')
!ls


In [ ]:
!pip install medspacy


In [1]:
import pandas as pd

In [6]:
with open("F:\C\Data\Hm2\\new1.txt") as f:
    text = f.read()

In [7]:
text

'XXXXXXXX   XXXXXXXXXXXXXXXXXX            XXXX XXXXXXXXX            XXXX XXXXXXX                  \nAge:   49 years     Sex:  Male     XXXX  XXXXXXXXXX \nAssociated Diagnoses:   None \nXXXXXXX   XXXXXXXXXXXXXXXXX XXXXXXXXX XXXXXXX XXXXXXXX \n \nXXXXX XXXXXXX \n \n\nXXXXXXXXXXXXXXXXXX                                 \nXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXX                                                                    \nXXXXXXXXXXXX            \n\nXXXXXXXXX XXX XXXX                 XXXXXXXXXXXX XXXXX XXXXXXXXX XXXXXXX         \nXXXXXXXXXXXXXXXXXXXX XXXXX XXXXXXXXXXXXXXXXXXX XXXXXXXX XXXXXXXXXXXXXXX           \n \nXXXXXXXXX XXXX  XXXXXXXXXX \nXXXXXXXXX XXX XXXX \n \n\nXX XXXXXXXX X XXXXXXXXX XXXXX XXXX X XXXXXXXXXX XXXXXXX XXXXXXXX \n            XXXXXXXXXXXXXX XXX XXXX X XXXXXXXX XXXXXX XX XXXXX XXXXXXXX \nXX XXXXXX XXX XXXX XXXX X XXXXXXXXXXX XXXXXXXXX XXX \nXXX XXXXXX  XXXX XXXXXXXX\n\nXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXXXXXX \nXX XXXXXXXX XXXXXXXXXXXXXXXXXXXXXXXXXXX \n            XXXXXXXX

In [ ]:
CATEGORY='Discharge summary'
HADM_ID=159647
CHARTDATE='2146/07/28'

In [8]:
import spacy;
import medspacy
# Option 1: Load default
nlp = medspacy.load()



In [9]:
sectionizer = nlp.add_pipe("medspacy_sectionizer")

In [10]:
nlp.pipe_names

['medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context',
 'medspacy_sectionizer']

In [11]:
doc = nlp(text)

In [12]:
len(doc._.sections)

9

In [13]:
doc._.section_bodies[3]


for Mr #####

1. Safe for discharge home today 

2. Nil new discharge medications
	Pain relief:
	- Please use simple analgesia - 1000mg paracetamol, four times a day, as needed
	- Follow up with your regular GP if you require further pain review
	
3. Follow up with regular GP within 3-5 days to review progress and medications 

4. Follow up with regular GP next Wednesday the #### of August for clips out

5. Follow up with Dr ######## within 14 days 
 	- Please call ######### 6921 to make an appointment 
 	- Address: #############, Campbelltown NSW 2560 

6. Mr #####, please seek medical attention if you have any symptoms (e.g. bleeding, increasing pain, fevers) or concerns

It has been pleasure taking care of Mr #####.

Kind regards,

############
Upper GI RMO 
on behalf of Dr. ######## 
Upper GI Consultant 
##################### 
*******************************************************
Operation Note 
 
Date of Operation: ########## . 
Surgeons 
Surgeon Incharge: FEDORINE, SERGUEI.  Op

In [14]:
doc._.section_titles[3]

Discharge plan

In [17]:
doc._.section_titles

[,
 Diagnoses:,
 discharge plan,
 Discharge plan,
 discharge plan,
 Discharge plan,
 ALLERGIES,
 Allergies,
 Indication :]

In [18]:
doc._.section_categories

[None,
 'diagnoses',
 'observation_and_plan',
 'observation_and_plan',
 'observation_and_plan',
 'observation_and_plan',
 'allergies',
 'allergies',
 'labs_and_studies']

In [19]:
doc._.section_spans

[XXXXXXXX   XXXXXXXXXXXXXXXXXX            XXXX XXXXXXXXX            XXXX XXXXXXX                  
 Age:   49 years     Sex:  Male     XXXX  XXXXXXXXXX 
 Associated,
 Diagnoses:   None 
 XXXXXXX   XXXXXXXXXXXXXXXXX XXXXXXXXX XXXXXXX XXXXXXXX 
  
 XXXXX XXXXXXX 
  
 
 XXXXXXXXXXXXXXXXXX                                 
 XXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXX                                                                    
 XXXXXXXXXXXX            
 
 XXXXXXXXX XXX XXXX                 XXXXXXXXXXXX XXXXX XXXXXXXXX XXXXXXX         
 XXXXXXXXXXXXXXXXXXXX XXXXX XXXXXXXXXXXXXXXXXXX XXXXXXXX XXXXXXXXXXXXXXX           
  
 XXXXXXXXX XXXX  XXXXXXXXXX 
 XXXXXXXXX XXX XXXX 
  
 
 XX XXXXXXXX X XXXXXXXXX XXXXX XXXX X XXXXXXXXXX XXXXXXX XXXXXXXX 
             XXXXXXXXXXXXXX XXX XXXX X XXXXXXXX XXXXXX XX XXXXX XXXXXXXX 
 XX XXXXXX XXX XXXX XXXX X XXXXXXXXXXX XXXXXXXXX XXX 
 XXX XXXXXX  XXXX XXXXXXXX
 
 XXXXXXX XXXXXXXXXXXXX XXXXXXXXXXXXXXX 
 XX XXXXXXXX XXXXXXXXXXXXXXXXXXXXXXXXXXX 
             XXXXXX

In [ ]:

import re
def preprocess(x):
    y=re.sub('\[(.*?)\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('Admission Date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('Discharge Date:','',y)
    y=re.sub('Date of Birth:','',y)
    y=re.sub('date of birth:','',y)
    y=re.sub('--|__|==','',y)
    y = y.strip()
    return y

In [ ]:

ls_notes=[]
for j in range(len(doc._.sections)):
          mystr=str(doc._.section_spans[j])
          mystr_Wothout_line=preprocess(mystr)
          ls_notes.append([HADM_ID,CHARTDATE,CATEGORY,mystr_Wothout_line,doc._.section_categories[j]])

df2=pd.DataFrame(ls_notes,columns=["HADM_ID",'CHARTDATE','CATEGORY',"body_span_Wothout_line",'category'])

df2

Step 2 : Finding CUI from each section text

In [ ]:
!pip install scispacy

In [ ]:
!pip install negspacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
CATEGORY='Discharge summary'
HADM_ID=159647
CHARTDATE='2146/07/28'

In [ ]:
import re

import scispacy
import spacy

from negspacy.negation import Negex
from negspacy.termsets import termset
from scispacy.linking import EntityLinker
import pandas as pd

#nlp = spacy.load("en_ner_bc5cdr_md")
nlp = spacy.load("en_core_web_sm")
#
ts = termset("en_clinical")
nlp.add_pipe("negex", config={"neg_termset": ts.get_patterns()})
nlp.add_pipe("scispacy_linker", config={"linker_name": "umls", "max_entities_per_mention": 1})

linker = nlp.get_pipe("scispacy_linker")
nlp.pipe_names

In [ ]:
nlp.pipe_names

In [ ]:
text='Pt. is a 76 yo female with a history of ESRD on HD, CAD,atrial fibrillation, and DM2 who presents with a 1 day history of 3 episodes of nonbloody, nonbilious, clear emesis and 1  episode of nonbloody diarrhea.'

In [ ]:
d = {'HADM_ID':159647, 'CHARTDATE': '2146/07/28','body_span_Wothout_line':text, 'category':'past_medical_history'}
df = pd.DataFrame(data=d,index=[0])
df

In [ ]:
list_Cui=[]
list_Exception=[]



for i in range(len(df)):
            text= str(df.iloc[i]['body_span_Wothout_line'])
            HADM_ID	=df.iloc[i]['HADM_ID']
            CHARTDATE	=df.iloc[i]['CHARTDATE']

            category_Inner=	df.iloc[i]['category']

            doc = nlp(text)


            print("\n")
            for entity in doc.ents:

                if(len(entity._.kb_ents)>0):
                    first_cuid = entity._.kb_ents[0][0]
                    kb_entry = linker.kb.cui_to_entity[first_cuid]
                    list_Cui.append([HADM_ID,CHARTDATE,category_Inner,entity._.negex,entity.text,
                                     first_cuid, kb_entry.canonical_name,entity.label_  ])
                else:
                    list_Exception.append([HADM_ID,CHARTDATE,category_Inner,entity.text])
                    continue



df_cui=pd.DataFrame(list_Cui,columns=['HADM_ID','CHARTDATE','category_Inner','negex','entity_text',
                                      'first_cuid', 'canonical_name','label'
                                          ])
df_cui